# 🚀 AI Ticket Support System (Final Version)

This notebook demonstrates a professional AI pipeline for customer support:
1. **Grouping**: Categorizing tickets.
2. **Confidence Scoring**: Detecting if a ticket is something we actually know how to answer.
3. **Answering (RAG)**: Finding the right solution.
4. **Web UI**: A live interface to test the AI.

### Step 1: Install Dependencies

In [ ]:
!pip install sentence-transformers scikit-learn numpy gradio

### Step 2: Load the AI Model

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import gradio as gr

print("Loading AI Model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model Loaded!")

### Step 3: Knowledge Base & Configuration

In [ ]:
knowledge_base = {
    "Password Reset": "To reset your password, go to the login page and click 'Forgot Password'. You will receive an email with a reset link.",
    "Leave Balance": "You can check your leave balance in the HR Portal under the 'My Benefits' section.",
    "Login Issues": "If your password is correct but you still can't log in, please clear your browser cache or contact IT support."
}

faq_topics = list(knowledge_base.keys())
faq_embeddings = model.encode(faq_topics)

# CONFIG: If the AI is less than 40% sure, it will say I don't know.
CONFIDENCE_THRESHOLD = 0.40

### Step 4: Define the Answering Logic

In [ ]:
def answer_ticket(user_ticket):
    # 1. Encode user ticket
    ticket_embedding = model.encode(user_ticket)
    
    # 2. Find closest FAQ
    scores = util.cos_sim(ticket_embedding, faq_embeddings)[0]
    best_match_idx = np.argmax(scores)
    confidence_score = float(scores[best_match_idx])
    
    topic = faq_topics[best_match_idx]
    
    # 3. Check Confidence
    if confidence_score < CONFIDENCE_THRESHOLD:
        return (
            f"UNCERTAIN (Score: {confidence_score:.2f})", 
            "I'm sorry, I'm not quite sure about that. Let me connect you to a human agent for better assistance."
        )
    
    solution = knowledge_base[topic]
    return (
        f"{topic} (Confidence: {confidence_score:.2f})", 
        f"Hello! {solution}"
    )

### Step 5: Launch the User Interface (Gradio)
Run this cell and a web link will appear!

In [ ]:
demo = gr.Interface(
    fn=answer_ticket,
    inputs=gr.Textbox(lines=2, placeholder="Type your support ticket here...", label="User Ticket"),
    outputs=[
        gr.Textbox(label="AI Topic Detection"),
        gr.Textbox(label="AI Suggested Response")
    ],
    title="🏢 AI Support Ticket Helpdesk",
    description="This AI automatically identifies the topic and provides answers using RAG. If it's unsure, it flags the ticket for a human.",
    examples=[
        "I forgot my password",
        "How many leaves do I have left?",
        "What is the color of the sky?" # Example of an 'unknown' ticket
    ]
)

demo.launch(debug=True, share=True)